<a href="https://colab.research.google.com/github/arnoldjames98/arnoldjames98.github.io/blob/main/systemKinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Biodegradable Seed Spreading Robot](https://arnoldjames98.github.io/)
# System Kinematics

### 1. Create a figure (either in python, in a vector-based drawing program like inkscape or illustrator, or as a solidworks rendering) of your system kinematics.

TODO: Describe figure and add correct measurements to table

![Mechanism Drawing](https://raw.githubusercontent.com/arnoldjames98/arnoldjames98.github.io/main/images/assignment3/mechanism.png)

| Length | Rigid Body | Measurement (cm)
| --- | --- | --- |
| L10 | Yellow | 2 |
| L02 | Purple | 2 |
| L42 | Purple | 2 |
| L40 | Purple | 2 |
| L31 | Red | 2 |
| L61 | Red | 2 |
| L63 | Red | 2 |
| L54 | Green | 2 |
| L32 | Orange | 2 |
| L56 | Blue | 2 |
| L5E | Blue | 2 |
| LE6 | Blue | 2 |


### 2. Make the device in paper or cardboard. You need an up-to-date model if it has changed from your individual assignments. The paper model should dimensionally match your code.

TODO: Upload image of device to github and add images here. May need to add a brief description too.

### 3. Using a pynamics-based script, develop a kinematic model for your device.

In [21]:
%matplotlib inline
!pip install pynamics

import all required packages

In [22]:
import pynamics
from pynamics.frame import Frame
from pynamics.variable_types import Differentiable,Constant
from pynamics.system import System
#from pynamics.body import Body
#from pynamics.dyadic import Dyadic
from pynamics.output import Output,PointsOutput
#from pynamics.particle import Particle
import pynamics.integration
import sympy
import numpy
import matplotlib.pyplot as plt
plt.ion()
from math import pi
#from pynamics.constraint import Constraint
import scipy.optimize

Create a pynamics system

In [23]:
system = System()
pynamics.set_system(__name__,system)

Declare all length constants

In [24]:
# TODO: Add correct measurements when defining length constants
# We may also considering our angle constraints for the triangular links here

# From Loop Closure Eq. #1
l_10 = Constant(2,'l_10',system)
l_31 = Constant(1.5,'l_31',system)
l_32 = Constant(1,'l_32',system)
l_02 = Constant(1,'l_02',system)

# From Loop Closure Eq. #2
#l_10 already is included in Eq. 1
l_61 = Constant(2,'l_61',system)
l_56 = Constant(2,'l_56',system)
l_54 = Constant(2,'l_54',system)
l_40 = Constant(2,'l_54',system)

# From Loop Closure Eq. #3
#l_32 already is included in Eq. 1
#l_54 already is included in Eq. 2
#l_56 already is included in Eq. 2
l_63 = Constant(2,'l_63',system)
l_42 = Constant(2,'l_42',system)

# End effector
l_E6 = Constant(2,'l_E6',system)

Create the differentiable state variables, cooresponding to the joint angles (we might not need all of these depending on how we do the constraints)

In [25]:
q0,q0_d,q0_dd = Differentiable('q0',system)
q1,q1_d,q1_dd = Differentiable('q1',system)
q2,q2_d,q2_dd = Differentiable('q2',system)
q3,q3_d,q3_dd = Differentiable('q3',system)
q4,q4_d,q4_dd = Differentiable('q4',system)
q5,q5_d,q5_dd = Differentiable('q5',system)
q6,q6_d,q6_dd = Differentiable('q6',system)

Initial guess for starting positions

In [26]:
# TODO: We can measure the angles on our figure to make some guesses

Retrieve state variables in the order they are stored in the system

In [27]:
# Not sure if we need this, but it's included in the tutorial
statevariables = system.get_state_variables()

Create the reference frames, with one cooresponding to each rigid link
![Reference Frames](https://raw.githubusercontent.com/arnoldjames98/arnoldjames98.github.io/main/images/assignment3/refFrames.png)

In [28]:
N = Frame('N') # Purple link at p0 (ground)
A = Frame('A') # Yellow link at p1
B = Frame('B') # Green link at p5
C = Frame('C') # Red link at p6
D = Frame('D') # Orange link at p3
E = Frame('E') # Blue link at pE (end-effector)

Declare N as the Newtonian (fixed) frame

In [29]:
system.set_newtonian(N)

Rotate frames about their z-axes so they align with the correct rigid link.

In [30]:
A.rotate_fixed_axis_directed(N,[0,0,1],q1,system)
B.rotate_fixed_axis_directed(A,[0,0,1],q5,system)
C.rotate_fixed_axis_directed(B,[0,0,1],q6,system)
D.rotate_fixed_axis_directed(B,[0,0,1],q3,system)

# The end effector frame is a bit different, so may need to reconsider this
# For now it's the same as frame D
E.rotate_fixed_axis_directed(B,[0,0,1],q3,system)

Define my rigid body kinematics